# Gov Covid API
### https://coronavirus.data.gov.uk/developers-guide

Install library with: `pip install uk-covid19`

In [1]:
import uk_covid19 as cv
import pandas as pd
import numpy as np

from datetime import date, timedelta

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline


In [2]:
filters = ["areaType=utla", "areaName=Nottinghamshire"] 

dataStructure = {   
    "date": "date", 
    "areaName": "areaName", 
    "newCases": "newCasesBySpecimenDate",  
    "newDeaths": "newDeathsByDeathDate", 
    } 

objCV19 = cv.Cov19API(filters=filters, structure=dataStructure) 

apiResult = objCV19.get_dataframe()

apiResult.head()

,date,areaName,newCases,newDeaths
0,2020-10-27,Nottinghamshire,1,2.0
1,2020-10-26,Nottinghamshire,334,2.0
2,2020-10-25,Nottinghamshire,261,7.0
3,2020-10-24,Nottinghamshire,260,2.0
4,2020-10-23,Nottinghamshire,383,6.0


### Data at local level is unreliable for yesterday so the following code removes it and ignores data older than 62 days.

In [3]:
earliestDate = date.today() - timedelta(days=62)
earliestDate = earliestDate.strftime('%Y-%m-%d')

yesterday = date.today() - timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')

df = apiResult

df = df[(df["date"] >= earliestDate) & (df["date"] < yesterday)]

df = df.sort_values(by="date")

### Calculate 7 day rolling averages for New Cases and New Deaths.

In [4]:
df['newCases_SMA_7'] = df["newCases"].rolling(window=7).mean()
df['newDeaths_SMA_7'] = df["newDeaths"].rolling(window=7).mean()

df.head(8)

,date,areaName,newCases,newDeaths,newCases_SMA_7,newDeaths_SMA_7
61,2020-08-27,Nottinghamshire,7,1.0,NaN,NaN
60,2020-08-28,Nottinghamshire,11,0.0,NaN,NaN
59,2020-08-29,Nottinghamshire,10,1.0,NaN,NaN
58,2020-08-30,Nottinghamshire,15,0.0,NaN,NaN
57,2020-08-31,Nottinghamshire,10,0.0,NaN,NaN
56,2020-09-01,Nottinghamshire,35,2.0,NaN,NaN
55,2020-09-02,Nottinghamshire,33,1.0,17.285714,0.714286
54,2020-09-03,Nottinghamshire,40,1.0,22.000000,0.714286


In [5]:
plt.figure(figsize=[15,4])
plt.grid(True)

ax = plt.gca()
locator = mdates.DayLocator(interval = 7)
ax.xaxis.set_major_locator(locator)

plt.xlabel = "Date"
plt.ylabel = "Cases"

plt.plot(df["date"], df["newCases"], label="New Cases")
plt.plot(df["date"], df["newCases_SMA_7"], label="7 Day Average of New Cases")
plt.legend(loc=2)

plt.show()

In [6]:
plt.figure(figsize=[15,4])
plt.grid(True)

ax = plt.gca()
locator = mdates.DayLocator(interval = 7)
ax.xaxis.set_major_locator(locator)

plt.xlabel = "Date"
plt.ylabel = "Deaths"

plt.plot(df["date"], df["newDeaths"], label="New Deaths")
plt.plot(df["date"], df["newDeaths_SMA_7"], label="7 Day Average of New Deaths")
plt.legend(loc=2)

plt.show()